# Tarea 2 – ISIS 4221, Notebook II

## Paso 0: Importación de Notebooks Necesarios

In [ ]:
%run "Notebook I.ipynb"

## Paso 1: Definición de las Funciones Necesarias

In [ ]:
def compute_perplexity(test_set, ngram_counts, final_unigram, n_gram):
    """
    Calcula la perplejidad de un conjunto de prueba usando el modelo de n-gramas.

    La perplejidad es una métrica que evalúa qué tan bien el modelo predice el texto de prueba. 
    Valores más bajos de perplejidad indican mejores predicciones.

    Args:
        test_set (list): El conjunto de oraciones de prueba.
        ngram_counts (dict): El modelo de n-gramas con las frecuencias de los n-gramas.
        final_unigram (dict): El modelo de (n-1)-gramas con las frecuencias.
        n_gram (int): El tamaño del n-grama (1 para unigramas, 2 para bigramas, etc.).

    Returns:
        float: El valor de la perplejidad calculada para el conjunto de prueba.
    """
    perplexity = 0  # Inicializa la suma acumulativa de la perplejidad.
    total_words = 0  # Contador para el número total de palabras procesadas.
    
    for sentence in test_set:
        tokens = sentence.split()  # Divide la oración en tokens.
        
        # Recorre los tokens desde el índice n_gram - 1 hasta el final.
        for i in range(n_gram - 1, len(tokens)):
            # Estima la probabilidad del n-grama actual.
            prob = estimate_probability(tokens[i-n_gram+1:i+1], n_gram, final_unigram, ngram_counts)
            # Calcula la logaritmo negativo de la probabilidad y la suma a la perplejidad.
            perplexity -= np.log(prob)
        
        # Incrementa el número total de palabras procesadas.
        total_words += len(tokens) - (n_gram - 1)
    
    # Devuelve la perplejidad exponencial media.
    return np.exp(perplexity / total_words)

def generate_sentence(start_word, ngram_counts, final_unigram, n_gram, max_length=15):
    """
    Genera una oración automáticamente a partir de un modelo de n-gramas.

    Esta función toma una palabra inicial y construye una oración basada en los n-gramas más frecuentes
    del modelo, generando hasta `max_length` palabras.

    Args:
        start_word (str): La primera palabra de la oración.
        ngram_counts (dict): El modelo de n-gramas con las frecuencias de los n-gramas.
        final_unigram (dict): El modelo de (n-1)-gramas con las frecuencias.
        n_gram (int): El tamaño del n-grama utilizado (1 para unigramas, 2 para bigramas, etc.).
        max_length (int): La longitud máxima de la oración generada.

    Returns:
        str: La oración generada automáticamente.
    """
    sentence = [start_word]  # Inicializa la oración con la palabra inicial.
    
    for _ in range(max_length):
        # Encuentra los candidatos a la siguiente palabra basados en el contexto actual.
        next_token_candidates = [(ngram[-1], count) for ngram, count in ngram_counts.items()
                                 if ngram[:-1] == tuple(sentence[-(n_gram-1):])]
        
        if not next_token_candidates:
            break  # Sale del bucle si no hay candidatos válidos.
        
        # Selecciona la siguiente palabra con la mayor frecuencia.
        next_token = max(next_token_candidates, key=lambda x: x[1])[0]
        sentence.append(next_token)  # Añade la palabra seleccionada a la oración.
        
        if next_token == '</s>':
            break  # Termina la oración si se encuentra el token de finalización.
    
    return ' '.join(sentence)  # Devuelve la oración generada como una cadena de texto.

## Paso 2: Cálculo de la Perplejidad de los Modelos de N-gramas de 20N y BAC

In [ ]:
# Calcula la perplejidad de los modelos de N-gramas de 20N
perplexity_unigram_20n = compute_perplexity(df_news_test['text'], unigram_counts_20n, unigram_20n, 1)
perplexity_bigram_20n = compute_perplexity(df_news_test['text'], bigram_counts_20n, bigram_20n, 2)
perplexity_trigram_20n = compute_perplexity(df_news_test['text'], trigram_counts_20n, trigram_20n, 3)

print(f"Perplejidad (Unigrama 20N): {perplexity_unigram_20n}")
print(f"Perplejidad (Bigrama 20N): {perplexity_bigram_20n}")
print(f"Perplejidad (Trigrama 20N): {perplexity_trigram_20n}")

In [ ]:
# Calcula la perplejidad de los modelos de N-gramas de BAC
perplexity_unigram_bac = compute_perplexity(df_bac_test['text'], unigram_counts_bac, unigram_bac, 1)
perplexity_bigram_bac = compute_perplexity(df_bac_test['text'], bigram_counts_bac, bigram_bac, 2)
perplexity_trigram_bac = compute_perplexity(df_bac_test['text'], trigram_counts_bac, trigram_bac, 3)

print(f"Perplejidad (Unigrama BAC): {perplexity_unigram_bac}")
print(f"Perplejidad (Bigrama BAC): {perplexity_bigram_bac}")
print(f"Perplejidad (Trigrama BAC): {perplejidad_trigram_bac}")

## Paso 3: Generación de Sentencias con los Mejores Modelos de N-gramas de 20N y BAC

<span style="color: red;">Lo que sigue es solo un ejemplo. Falta encontrar los mejores modelos de cada conjunto de datos.</span>

In [ ]:
# Ejemplo de generación de oraciones con `bigram_counts_20n`
generated_sentence_20n = generate_sentence('<s>', bigram_counts_20n, bigram_20n, 2)
print(f"Oración generada (20N): {generated_sentence_20n}")

In [ ]:
# Ejemplo de generación de oraciones con `bigram_counts_bac`
generated_sentence_bac = generate_sentence('<s>', bigram_counts_bac, bigram_bac, 2)
print(f"Oración generada (BAC): {generated_sentence_bac}")